In [1]:
import os
import subprocess
import shlex
import sys
import json
from IPython.display import clear_output
from ipynb.fs.full.cs_filepaths import FilePaths

### Command

In [2]:
class Command:
    file_paths = FilePaths()
    
    def __init__(self, command):
        self.command = command
        self.stdout = b""
        self.stderr = b""
    
    def execute(self):
        print(f"Executing: {self.command}")
        process = subprocess.Popen(
#             shlex.split(self.command),
            self.command,
            stdout=subprocess.PIPE, 
            stderr=subprocess.PIPE,
            shell=True
        )
        stdout, stderr = process.communicate()
        self.stdout = stdout
        self.stderr = stderr
        if self.has_error():
            raise Exception(self.err_as_str())
        clear_output(wait=False)
        return self
    
    def has_error(self):
        return len(self.stderr) > 0
    
    def out_as_bytes(self):
        return self.stdout
    
    def err_as_bytes(self):
        return self.stderr
    
    def out_as_str(self):
        return self.stdout.decode("utf-8")
    
    def err_as_str(self):
        return self.stderr.decode("utf-8")
    
    def write_out_to_file(self, file_name):
        with open(file_name, 'wb') as f: 
            f.write(self.out_as_bytes())
        return self
    
    def out_file(self):
        raise Exception("Method not implemented")

In [3]:
class GitCheckoutCommand(Command):
    def __init__(self, git_dir, commit):
        command_str = f"git -C {git_dir} checkout {commit}"
        super().__init__(command_str)
    
    def execute(self):
        # I don't know why but executing git checkout writes to stderr
        try:
            super().execute()
        except Exception as e:
            if not "HEAD is now at" in str(e):
                raise(e)
            else:
                pass
        return self

In [13]:
class GitLogCommand(Command):
    def __init__(self, git_file, before=None, after=None):
        command_str = f"git --git-dir {git_file} \
            log --pretty=format:'[%h] %an %ad %s' --date=short --numstat"
        if after:
            command_str = command_str + f" --after={after}"
        if before:
            command_str = command_str + f" --before={before}"
        super().__init__(command_str)
    
    def execute(self):
        super().execute()
        # Raise exception if no log output was generated
        if len(self.out_as_bytes()) == 0:
            raise Exception("No log output generated")
        return self
    
    def write_out_to_file(self):
        return super().write_out_to_file(self.out_file())
    
    def out_file(self):
        return self.file_paths.log_file

In [5]:
class GetLatestCommitShaBeforeDate(Command):
    def __init__(self, git_file, before=None):
        command = f"git --git-dir {git_file} rev-list -n 1 HEAD"
        if before is not None:
            command = f"{command} --before='{before}'"
        super().__init__(command)
        
    def execute(self):
        super().execute()
        # Raise exception if no log output was generated
        if len(self.out_as_bytes()) == 0:
            raise Exception("No log output generated")
        return self
    
class GetEarliestCommitShaAfterDate(Command):
    def __init__(self, git_file, after=None):
        command = f"git --git-dir {git_file} log --reverse --pretty=format:%H"
        if after is not None:
            command = f"{command} --after='{after}'"
        command = f"{command} | head -n 1"
        print(command)
        super().__init__(command)
        
    def execute(self):
        super().execute()
        # Raise exception if no log output was generated
        if len(self.out_as_bytes()) == 0:
            raise Exception("No log output generated")
        return self

In [6]:
class MaatCommand(Command):
    def __init__(self, args_str):
        super().__init__(f"java -jar {self.file_paths.codemaat_jar} {args_str}")
        
    def write_out_to_file(self):
        return super().write_out_to_file(self.out_file())
    
    def out_file(self):
        return self.file_paths.maat_output_csv

In [7]:
class ClocCommand(Command):
    def __init__(self, args_str):
        super().__init__(f"cloc {args_str}")
        
    def write_out_to_file(self):
        return super().write_out_to_file(self.out_file())
    
    def out_file(self):
        return self.file_paths.cloc_lines_csv

In [8]:
class MergeComplexityAndFrequency(Command):
    def __init__(self, freqs_file, lines_file):
        command = f"python \
            {self.file_paths.cs_scripts}/merge_comp_freqs.py \
            {freqs_file} \
            {lines_file}"
        super().__init__(command)

In [10]:
class FileComplexityCommand(Command):
    def __init__(self, file_path):
        command = f"python {self.file_paths.cs_scripts}/complexity_analysis.py {file_path}"
        super().__init__(command)

In [11]:
class FileComplexityTrendCommand(Command):
    def __init__(self, git_dir, relative_file_path, start_commit, end_commit):
        command = f"python {self.file_paths.cs_scripts}/git_complexity_trend.py \
            --git-dir {git_dir} \
            --file {relative_file_path} \
            --start {start_commit} \
            --end {end_commit}"
        super().__init__(command)

In [9]:
class CreateEnclosureDiagramJson(Command):
    def __init__(self, freqs_file, lines_file, weight_column=1):
        command = f"python {self.file_paths.cs_scripts}/csv_as_enclosure_json.py \
            --structure {lines_file} \
            --weights {freqs_file} \
            --weightcolumn {weight_column}"
        super().__init__(command)

In [12]:
class CreateMainDevEnclosureDiagramJson(Command):
    def __init__(self, lines_file, main_devs_file, author_colours_file):
        command = f"python {self.file_paths.cs_scripts}/csv_main_dev_as_knowledge_json.py \
            --structure {lines_file} \
            --owners {main_devs_file} \
            --authors {author_colours_file}"
        super().__init__(command)